In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Layer, Conv2D, Input
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [4]:
# Hyperparameters
IMAGE_SIZE = 128
BATCH_SIZE = 16
NUM_EPOCHS = 10
NUM_CHANNELS = 5
NUM_PC_OUTPUTS = NUM_CHANNELS * 36
STEPS_PER_EPOCH = 500
NUM_CLASSES = 2  

In [18]:

data_dir = "data"
test_dir = "CapsNet/data/test"
train_dir = "CapsNet/data/train"

In [14]:

train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='categorical', subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)


Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
# Capsule Layers
class PrimaryCapsule(Layer):
    def call(self, inputs):
        s_norm = tf.norm(inputs, axis=-1, keepdims=True)
        return (s_norm ** 2 / (1 + s_norm ** 2)) * (inputs / (s_norm + 1e-7))

class DigitCapsule(Layer):
    def call(self, inputs):
        s = tf.reduce_sum(inputs, axis=1)
        return PrimaryCapsule()(s)

class Mapping(Layer):
    def __init__(self):
        super().__init__()
        self.W = tf.Variable(tf.random.normal([NUM_PC_OUTPUTS, NUM_CLASSES, 8, 16]))

    def call(self, inputs):
        return tf.matmul(inputs, self.W)

In [16]:

# CapsNet Model
def CapsNet():
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = Conv2D(256, kernel_size=9, strides=1, activation='relu')(inputs)
    x = Conv2D(128, kernel_size=9, strides=2, activation='relu')(x)
    
    # Debugging: Print shape before reshaping
    print("Shape before reshape:", x.shape)
    
    x_shape = tf.shape(x)
    x = tf.reshape(x, [x_shape[0], -1, 8]) 
    
    primary_caps = PrimaryCapsule()(x)
    digit_caps_inputs = Mapping()(primary_caps)
    outputs = DigitCapsule()(digit_caps_inputs)
    model = Model(inputs, outputs)
    return model

model = CapsNet()
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])





Shape before reshape: (None, 56, 56, 128)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=NUM_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH)

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")